# dependencies
connect to cpu if want to use behaviour cloning

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 10.0 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install seals
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/82

# BC train on policy then give it to gail - test if it works

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms import bc
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(),
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/CartPole-v0",
    venv=env,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-seals-CartPole-v0.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
rng = np.random.default_rng(0)
rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=10),
    rng=rng,
)
transitions = rollout.flatten_trajectories(rollouts)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
from typing import Callable

from stable_baselines3 import PPO
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
policy_shared = MlpPolicy(observation_space = env.observation_space,
                          action_space = env.action_space,
                          lr_schedule = linear_schedule(1e-3))
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    policy = policy_shared,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [10]:
bc_trainer2 = bc.BC(
    observation_space=env.observation_space,
    policy = bc_trainer.policy,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [11]:
bc_trainer2.train(n_epochs=10)

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 137       |
|    loss           | 0.692     |
|    neglogp        | 0.693     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


249batch [00:04, 25.70batch/s]
498batch [00:12, 66.96batch/s]
Epoch 1 of 10                 

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000318 |
|    entropy        | 0.318     |
|    epoch          | 2         |
|    l2_loss        | 0         |
|    l2_norm        | 169       |
|    loss           | 0.49      |
|    neglogp        | 0.491     |
|    prob_true_act  | 0.783     |
|    samples_so_far | 16032     |
---------------------------------


739batch [00:14, 150.71batch/s]
992batch [00:15, 149.14batch/s]
Epoch 3 of 10                  

...

1238batch [00:17, 160.63batch/s]
1499batch [00:18, 167.53batch/s]
Epoch 5 of 10                   

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1500      |
|    ent_loss       | -0.000353 |
|    entropy        | 0.353     |
|    epoch          | 6         |
|    l2_loss        | 0         |
|    l2_norm        | 179       |
|    loss           | 0.232     |
|    neglogp        | 0.232     |
|    prob_true_act  | 0.821     |
|    samples_so_far | 48032     |
---------------------------------


1746batch [00:22, 41.42batch/s]
1999batch [00:24, 152.74batch/s]
Epoch 7 of 10                   

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 2000      |
|    ent_loss       | -0.000294 |
|    entropy        | 0.294     |
|    epoch          | 8         |
|    l2_loss        | 0         |
|    l2_norm        | 181       |
|    loss           | 0.348     |
|    neglogp        | 0.348     |
|    prob_true_act  | 0.79      |
|    samples_so_far | 64032     |
---------------------------------


2241batch [00:26, 141.85batch/s]
2484batch [00:28, 156.29batch/s]
2500batch [00:28, 88.30batch/s]


In [12]:
bc_trainer.train(n_epochs=50)
reward, _ = evaluate_policy(bc_trainer.policy, env, 10)
print("Reward:", reward)

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000336 |
|    entropy        | 0.336     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 181       |
|    loss           | 0.428     |
|    neglogp        | 0.429     |
|    prob_true_act  | 0.768     |
|    samples_so_far | 32        |
---------------------------------


236batch [00:01, 148.73batch/s]
489batch [00:03, 160.62batch/s]
Epoch 1 of 100                 

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000311 |
|    entropy        | 0.311     |
|    epoch          | 2         |
|    l2_loss        | 0         |
|    l2_norm        | 182       |
|    loss           | 0.43      |
|    neglogp        | 0.431     |
|    prob_true_act  | 0.743     |
|    samples_so_far | 16032     |
---------------------------------


746batch [00:04, 136.00batch/s]
997batch [00:07, 108.83batch/s]
Epoch 3 of 100                 

...

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 24000     |
|    ent_loss       | -0.000312 |
|    entropy        | 0.312     |
|    epoch          | 96        |
|    l2_loss        | 0         |
|    l2_norm        | 191       |
|    loss           | 0.43      |
|    neglogp        | 0.43      |
|    prob_true_act  | 0.76      |
|    samples_so_far | 768032    |
---------------------------------


24240batch [03:07, 102.40batch/s]
24489batch [03:09, 153.08batch/s]
Epoch 97 of 100                  

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 24500     |
|    ent_loss       | -0.000245 |
|    entropy        | 0.245     |
|    epoch          | 98        |
|    l2_loss        | 0         |
|    l2_norm        | 191       |
|    loss           | 0.265     |
|    neglogp        | 0.266     |
|    prob_true_act  | 0.855     |
|    samples_so_far | 784032    |
---------------------------------


24745batch [03:11, 152.90batch/s]
24994batch [03:12, 163.17batch/s]
25000batch [03:12, 129.76batch/s]


Reward: 500.0


In [13]:
trained_policy = bc_trainer.policy

In [14]:
import torch
torch.save(trained_policy, "PPO-Cartpolevo_1.pth.tar")

In [15]:
load_from_saved_policy = torch.load("PPO-Cartpolevo_1.pth.tar")

In [16]:
load_from_saved_policy.state_dict()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-4.8713e-02, -1.5184e-01, -1.2080e+00, -3.5663e-01],
                      [-1.5378e-02, -3.1764e-02, -1.9077e-01,  1.2898e-02],
                      [ 3.1269e-02, -5.3967e-02, -1.0688e+00, -4.1962e-01],
                      [-2.4390e-02, -4.0738e-02, -1.6173e-02, -6.1845e-02],
                      [-4.7808e-02, -6.4145e-02, -3.4068e-01, -7.3623e-02],
                      [ 1.6229e-02,  5.2451e-02,  4.0508e-01,  2.7613e-02],
                      [ 1.1561e-01,  2.3545e-01,  1.2854e+00,  5.9690e-01],
                      [ 2.4197e-02, -2.5100e-01, -1.1975e+00, -6.7842e-01],
                      [ 7.5086e-03,  5.1718e-02,  2.1507e-01,  2.3363e-02],
                      [-9.3435e-02, -5.2306e-02, -6.8840e-01, -5.0963e-01],
                      [ 4.1886e-02,  7.9714e-02,  2.5915e-01,  9.4281e-02],
                      [ 1.0672e-01,  1.2378e-01,  1.0423e+00,  5.4375e-01],
                      [ 2.3247e-04, -

In [17]:
trained_policy.state_dict()

OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-4.8713e-02, -1.5184e-01, -1.2080e+00, -3.5663e-01],
                      [-1.5378e-02, -3.1764e-02, -1.9077e-01,  1.2898e-02],
                      [ 3.1269e-02, -5.3967e-02, -1.0688e+00, -4.1962e-01],
                      [-2.4390e-02, -4.0738e-02, -1.6173e-02, -6.1845e-02],
                      [-4.7808e-02, -6.4145e-02, -3.4068e-01, -7.3623e-02],
                      [ 1.6229e-02,  5.2451e-02,  4.0508e-01,  2.7613e-02],
                      [ 1.1561e-01,  2.3545e-01,  1.2854e+00,  5.9690e-01],
                      [ 2.4197e-02, -2.5100e-01, -1.1975e+00, -6.7842e-01],
                      [ 7.5086e-03,  5.1718e-02,  2.1507e-01,  2.3363e-02],
                      [-9.3435e-02, -5.2306e-02, -6.8840e-01, -5.0963e-01],
                      [ 4.1886e-02,  7.9714e-02,  2.5915e-01,  9.4281e-02],
                      [ 1.0672e-01,  1.2378e-01,  1.0423e+00,  5.4375e-01],
                      [ 2.3247e-04, -

In [19]:
learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
learner.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

<All keys matched successfully>

In [ ]:
learner.policy.state_dict()

In [20]:
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)

In [21]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    gail_trainer, env, 10, return_episode_rewards=True
)
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

Rewards before training: 500.0 +/- 0.0


In [22]:
learner2 = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)

In [23]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner2, env, 10, return_episode_rewards=True
)
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

Rewards before training: 111.3 +/- 29.21318195609646


In [26]:
gail_trainer.train(50000)

round:   0%|          | 0/3 [00:00<?, ?it/s]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 500          |
|    gen/rollout/ep_rew_wrapped_mean | 309          |
|    gen/time/fps                    | 2625         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 6            |
|    gen/time/total_timesteps        | 32768        |
|    gen/train/approx_kl             | 0.0010066275 |
|    gen/train/clip_fraction         | 0.0208       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.299       |
|    gen/train/explained_variance    | 0.000532     |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | 0.21         |
|    gen/train/n_updates             | 5            |
|    gen/train/policy_gradient_loss  | 0.000131     |
|    gen/train/value_loss   

round:  33%|███▎      | 1/3 [00:14<00:29, 14.75s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 500          |
|    gen/rollout/ep_rew_wrapped_mean | 316          |
|    gen/time/fps                    | 2854         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 49152        |
|    gen/train/approx_kl             | 0.0030711936 |
|    gen/train/clip_fraction         | 0.037        |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.291       |
|    gen/train/explained_variance    | -0.046       |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | 0.119        |
|    gen/train/n_updates             | 10           |
|    gen/train/policy_gradient_loss  | -0.00187     |
|    gen/train/value_loss   

round:  67%|██████▋   | 2/3 [00:28<00:14, 14.06s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 500         |
|    gen/rollout/ep_rew_wrapped_mean | 322         |
|    gen/time/fps                    | 2986        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 65536       |
|    gen/train/approx_kl             | 0.003677166 |
|    gen/train/clip_fraction         | 0.0562      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.29       |
|    gen/train/explained_variance    | 0.415       |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.037       |
|    gen/train/n_updates             | 15          |
|    gen/train/policy_gradient_loss  | -0.00428    |
|    gen/train/value_loss            | 0.0932 

round: 100%|██████████| 3/3 [00:41<00:00, 13.88s/it]


# keeping records

In [ ]:
learner.policy.state_dict()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-4.4041e-02,  3.0514e-01,  2.4969e-01,  2.2949e-02],
                      [ 1.3385e-03,  1.0555e-01,  2.3079e-01, -2.3341e-03],
                      [-3.5703e-01,  4.6072e-01, -9.5808e-02,  1.0105e-02],
                      [-2.0877e-01,  8.9966e-03, -1.7518e-01,  2.8332e-01],
                      [-7.1176e-02,  4.2719e-02,  1.2800e-01,  1.3782e-01],
                      [ 2.3020e-01,  2.8346e-01,  6.7867e-02,  1.9514e-01],
                      [-6.2821e-02,  2.3852e-01, -3.5208e-01,  3.3932e-03],
                      [-2.0429e-01,  2.6291e-01,  2.1719e-02, -1.7294e-01],
                      [ 2.9023e-01,  6.3187e-02, -4.0027e-02,  1.0530e-01],
                      [ 1.2927e-01,  1.7725e-01,  2.6417e-01, -2.6777e-01],
                      [-7.4867e-02, -1.7507e-01, -3.5271e-01,  5.4199e-02],
                      [-6.3071e-03, -2.1550e-01, -1.9196e-01,  2.2225e-02],
                      [ 3.1760e-01, -